In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np

In [3]:
datadir = '../Collected Data/preprocessed/'

In [4]:
df = pd.read_csv(datadir+'preprocessed_data.csv')
df.head()

,avg_accX,min_accX,max_accX,var_accX,avg_accY,min_accY,max_accY,var_accY,avg_accZ,min_accZ,...,var_accZ,avg_magnitude,min_magnitude,max_magnitude,var_magnitude,avg_pressure,min_pressure,max_pressure,var_pressure,label
0,-1.001886,-1.055542,-0.997314,0.000078,0.129964,0.113281,0.181152,0.000192,-0.012829,-0.026855,...,0.000195,1.010539,1.005311,1.071143,0.000098,972.634906,972.628618,972.643121,0.000014,N
1,-0.671724,-1.251709,-0.221558,0.147100,0.582546,0.181152,0.857910,0.042282,-0.175347,-0.472534,...,0.042240,1.009968,0.739978,1.305443,0.028850,972.902315,972.643121,973.129535,0.040889,N
2,-0.397659,-0.471558,-0.273193,0.001757,0.888291,0.778931,0.992798,0.002689,-0.085164,-0.458252,...,0.074113,1.015058,0.954359,1.076219,0.001113,972.595858,972.498928,972.808610,0.010962,N
3,-0.321914,-0.505981,-0.210938,0.011940,0.961016,0.715332,1.166748,0.019437,0.168530,-0.130005,...,0.028908,1.049457,0.849687,1.216070,0.013574,972.587867,972.520764,972.892723,0.009303,N
4,-0.437627,-1.404663,-0.036377,0.176771,0.702866,0.223267,1.049805,0.061700,-0.218762,-0.548462,...,0.056466,0.984485,0.594284,1.474214,0.054309,972.609129,972.441795,972.892723,0.025621,N


In [18]:
X = df[['avg_accX','min_accX','max_accX','var_accX','avg_accY','min_accY','max_accY','var_accY','avg_accZ','min_accZ','max_accZ','var_accZ','avg_magnitude','min_magnitude','max_magnitude','var_magnitude','avg_pressure','min_pressure','max_pressure','var_pressure']].values

In [20]:
X[0]


array([-1.00188628e+00, -1.05554199e+00, -9.97314453e-01,  7.82857598e-05,
        1.29964193e-01,  1.13281250e-01,  1.81152344e-01,  1.91566905e-04,
       -1.28290086e-02, -2.68554688e-02,  2.05078125e-02,  1.94650459e-04,
        1.01053900e+00,  1.00531052e+00,  1.07114318e+00,  9.80876687e-05,
        9.72634906e+02,  9.72628618e+02,  9.72643121e+02,  1.41661535e-05])

In [21]:
y = df['label'].values

In [22]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [29]:
torch.manual_seed(42)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [30]:
X_tensor = torch.tensor(X, dtype=torch.float32).to(device)
y_tensor = torch.tensor(y_encoded, dtype=torch.long).to(device)

In [31]:
X_tensor, y_tensor

(tensor([[-1.0019e+00, -1.0555e+00, -9.9731e-01,  ...,  9.7263e+02,
           9.7264e+02,  1.4166e-05],
         [-6.7172e-01, -1.2517e+00, -2.2156e-01,  ...,  9.7264e+02,
           9.7313e+02,  4.0889e-02],
         [-3.9766e-01, -4.7156e-01, -2.7319e-01,  ...,  9.7250e+02,
           9.7281e+02,  1.0962e-02],
         ...,
         [-2.0689e-01, -2.2705e-01, -1.8311e-01,  ...,  9.7318e+02,
           9.7324e+02,  3.7111e-04],
         [-1.6399e-01, -1.9983e-01, -1.2549e-01,  ...,  9.7318e+02,
           9.7326e+02,  5.4642e-04],
         [-1.3223e-01, -1.3647e-01, -1.2061e-01,  ...,  9.7320e+02,
           9.7323e+02,  1.1300e-04]], device='cuda:0'),
 tensor([2, 2, 2,  ..., 2, 2, 2], device='cuda:0'))

In [32]:
dataset = TensorDataset(X_tensor, y_tensor)
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)

In [42]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1])  # Use the last output of the sequence
        return out

In [36]:
input_size = 20  # Number of features (accX, accY, accZ, Magnitude, Pressure)
hidden_size = 32
num_layers = 2
num_classes = 5  # Number of output classes

In [43]:
model = LSTMModel(input_size, hidden_size, num_layers, num_classes).to(device)

In [38]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [44]:
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    for batch_x, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x32 and 64x5)